what do you wanna do with the training dataset? and val. well for training we need at least 2 books to be in the input and then we mask one and find it using the other two representations.

also for validation. the columns for with we have history == Nan or that the history has items less than 2 we will remove them as well.

as for testing, we will again remove items for which we don't have at least two items in the history.
since the train data is already processed and is loaded, extract the users for which we don't have enough purchase history and get their user ids to remove from the training set.

as for the validation set, you can check the column history.

In [ ]:
np.sum([len(x.split())<2 for x in valdata.history]) # for each user in valdata, we at least have 2 items in the purchase history

NameError: name 'np' is not defined

In [ ]:
# Group the traindata which is the history data for each user
grouped_trial_train = traindata.groupby('user_id').agg({
    'parent_asin': lambda x: list(x)
}).reset_index()
np.sum([len(x)<3 for x in grouped_trial_train.parent_asin]) # so for the training set we also have at least 3 items, 2 will be masked.

0

In [1]:
import pandas as pd
import numpy as np
import itertools
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
import random
import torch
import torch.nn as nn
import re
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [23]:
# Import the data
traindata = pd.read_csv('processed_train.csv')
valdata = pd.read_csv('processed_valid.csv')
testdata = pd.read_csv('processed_test.csv')

In [24]:
traindata

,user_id,parent_asin,rating,timestamp,history,title_review,text,asin,title_meta,author,categories,description,average_rating,price,store,details
0,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,048643107X,5.0,1481688109000,NaN,Five Stars,Great resource on history of fair Isle knitti...,048643107X,Traditional Fair Isle Knitting (Dover Knitting...,Sheila McGregor,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...",Situated far to the north of the Scottish main...,4.7,18.95,Sheila McGregor (Author),{'Publisher': 'Dover Publications (September 1...
1,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1584799390,5.0,1481688355000,048643107X,Good time.,Gorgeous book. There are a few patterns that I...,1584799390,My Grandmother's Knitting: Family Stories and ...,Larissa Brown,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...",When most people think of their grandmother’s ...,4.2,11.99,"Larissa Brown (Author), Michael Crouser (Phot...","{'Publisher': 'Stewart, Tabori and Chang (Sept..."
2,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1606600478,5.0,1481688478000,048643107X 1584799390,Arrived in great time. Plan to make a few of t...,Gorgeous book. Well packed. Arrived in great t...,1606600478,Tudor Roses,Alice Starmore,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...","Part fashion, part photography, part English h...",4.7,53.83,Alice Starmore (Author),"{'Publisher': 'Calla Editions; Revised, Expand..."
3,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1596684372,5.0,1482218160000,048643107X 1584799390 1606600478,Five Stars,Good resource for simple fair isle patterns.,1596684372,200 Fair Isle Motifs: A Knitter's Directory,Mary Jane Mucklestone,"['Books', 'Arts & Photography', 'Graphic Design']",The comprehensive guide to Fair Isle knitting ...,4.8,16.49,Mary Jane Mucklestone (Author),{'Publisher': 'Interweave; Illustrated edition...
4,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,0918804973,5.0,1482218794000,048643107X 1584799390 1606600478 1596684372,Good book.,Alice Starmore is one of my favorite Fair Isle...,0918804973,Alice Starmore's Book of Fair Isle Knitting,Alice Starmore,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...",Scotland's Fair Isle has long been known for i...,4.8,47.7,Alice Starmore (Author),{'Publisher': 'Taunton Press; Reprint edition ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208939,AGH7KQD653ALUVNARQ7WD6YFJPNA,0997560207,5.0,1466519703000,0814436439,One of the Best Sales Leadership Books from On...,"David Brock is a brilliant sales expert, one o...",0997560207,Sales Manager Survival Guide: Lessons From Sal...,Mr. David A Brock,"['Books', 'Business & Money', 'Marketing & Sal...","Finally! The definitive guide to the toughest,...",4.6,18.95,Mr. David A Brock (Author),{'Publisher': 'Partners In EXCELLENCE; 1st edi...
208940,AGH7KQD653ALUVNARQ7WD6YFJPNA,1626343888,5.0,1493383948000,0814436439 0997560207,"like ""fighting fires"" and swooping in as the ""...",The Sales Manager role is arguably one of the ...,1626343888,The Sales Manager's Guide to Greatness: Ten Es...,Kevin F. Davis,"['Books', 'Business & Money', 'Management & Le...","2018 Axiom Business Book Award Winner, Silver ...",4.5,24.95,Kevin F. Davis (Author),{'Publisher': 'Greenleaf Book Group Press (Mar...
208941,AFVMMNIFDICHHSD5DOLM46TN4CZA,1483365751,5.0,1455464856000,NaN,A Must Read for Transformative Educators,I've been an educator for nearly 20 years. The...,1483365751,UnCommon Learning: Creating Schools That Work ...,Eric C. Sheninger,"['Books', 'Education & Teaching', 'Schools & T...",UnCommon Learning techniques set the stage for...,4.5,11.99,Eric C. Sheninger (Author),{'Publisher': 'Corwin; 1st edition (November 1...
208942,AFVMMNIFDICHHSD5DOLM46TN4CZA,1119244560,5.0,1494619067000,1483365751,Sheninger and Rubin give the blueprint for suc...,Sheninger and Rubin give the blueprint for su...,1119244560,"BrandED: Tell Your Story, Build Relationships,...",Eric C. Sheninger,"['Books', 'Education & Teaching', 'Schools & T...","Praise for BrandED ""A great resource for educa...",4.6,NaN,"Trish Rubin (Author), Eric

In [25]:
DECAY_RATE = 0.1
HIDDEN_SIZE = 120 #768
NUM_ATTN_HEADS =2
NUM_ENC_LAYERS = 6
FEED_FORWARD_NEURONS = 120# 3072
DROPOUT_RATE = 0.1
MAX_LENGTH = 12 # the max length of the items in purchase history
MAX_ITEMS = 20000
unk_token = '<UNK>'
PAD_TOKEN = '<PAD>'
MASK_TOKEN = MAX_ITEMS - 1
BATCH_SIZE = 20
EPOCHS = 50

In [26]:
# Extract all the unique authors in the dataset
train_authors = [item.replace(' ', '').lower() for item in traindata['author'] if pd.notna(item)]
val_authors = [item.replace(' ', '').lower() for item in valdata['author'] if pd.notna(item)]
test_authors = [item.replace(' ', '').lower() for item in testdata['author'] if pd.notna(item)]
unique_authors = np.unique(train_authors + val_authors + test_authors)

def preprocess_authors(author):
    # Step 1: Extract the name (remove everything before and including the parentheses)
    name = re.sub(r'^\(.*\)', '', author)

    # Step 2: Replace dots with underscores
    name = name.replace('.', '')

    return name

# Apply the function to extract names
extracted_authors = [preprocess_authors(author) for author in unique_authors]
extracted_authors[0:10]

['ariel',
 'annareynolds',
 'ruthbailey',
 'deewilliams',
 'timothyhoward',
 'dominiqueansel',
 'normanking',
 'pipjones',
 'tonycliff',
 'richardwarburton']

In [27]:
import numpy as np

# Extract the parent_asins from each dataset (testdata, traindata, valdata)
#parent_asins_test = [item for item in testdata['parent_asin']]
parent_asins_train = [item for item in traindata['parent_asin']]
#parent_asins_val = [item for item in valdata['parent_asin']]

# Combine all parent_asins from all datasets
#parent_asins = parent_asins_test + parent_asins_train + parent_asins_val

# Extract the history from each dataset and split by spaces (if it's a string)
#history_test = [item for sublist in testdata['history'].apply(lambda x: x.split() if isinstance(x, str) else []).tolist() for item in sublist]
history_train = [item for sublist in traindata['history'].apply(lambda x: x.split() if isinstance(x, str) else []).tolist() for item in sublist]
#history_val = [item for sublist in valdata['history'].apply(lambda x: x.split() if isinstance(x, str) else []).tolist() for item in sublist]

# Combine all history data from the datasets
#all_history = history_test + history_train + history_val

# Combine history and parent_asins and then get unique book_ids
#all_book_ids = all_history + parent_asins
unique_book_ids = np.unique(parent_asins_train + history_train)

# Length of unique book_ids
len(unique_book_ids)


128730

In [29]:
# Extract all the user ids:
unique_user_ids = traindata.user_id.unique()

In [30]:
# Define a tokenizer for all the items purchased.
Tokenizer_book_id = Tokenizer()
Tokenizer_book_id.fit_on_texts(unique_book_ids)
Tokenizer_book_id.word_index['unk/pad'] = 0
# Define a Tokenizer for the authors
Tokenizer_author = Tokenizer()
Tokenizer_author.fit_on_texts(extracted_authors)
Tokenizer_author.word_index['unk/pad'] = 0
# Define a Tokenizer for the user ids
Tokenizer_user_id = Tokenizer()
Tokenizer_user_id.fit_on_texts(unique_user_ids)
Tokenizer_user_id.word_index['unk/pad'] = 0

In [ ]:
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer
from tensorflow.keras.preprocessing.text import Tokenizer

def process_dataset(dataset):
    # Initialize the SentenceTransformer
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # Tokenize the items purchased (parent_asin)
    dataset['item_embedded'] = Tokenizer_book_id.texts_to_sequences(dataset['parent_asin'])

    # Handle genres (convert lists to strings and fill NaN)
    dataset['categories'] = dataset['categories'].apply(lambda x: ' '.join(x) if isinstance(x, list) else 'NaN')
    dataset['categories'] = dataset['categories'].fillna('NaN')

    # Create an embedding the genres
    print("Generating genre embeddings...")
    genre_embeddings = []
    for genre in tqdm(dataset['categories'], desc="Genre Embedding Progress"):
        genre_embeddings.append(embedder.encode(genre))
    dataset['genres_embedded'] = genre_embeddings

    # Tokenize the authors (clean spaces, lowercase, and handle missing values)
    dataset['author'] = [author.replace(' ', '').lower() if pd.notna(author) else 'unk' for author in dataset['author']]

    # Tokenize
    dataset['author_tokenized'] = Tokenizer_author.texts_to_sequences(dataset['author'])

    # Tokenize the descriptions (handle missing descriptions and apply SentenceTransformer)
    dataset['description'] = dataset['description'].apply(lambda x: x if pd.notna(x) else 'No description')

    # Create a progress bar for embedding the descriptions
    print("Generating description embeddings...")
    description_embeddings = []
    for description in tqdm(dataset['description'], desc="Description Embedding Progress"):
        description_embeddings.append(embedder.encode(description))
    dataset['description_embedded'] = description_embeddings

    return dataset

# Apply the processing function to the training data
traindata = process_dataset(traindata)

# Save the DataFrames to pickle files
with open('train_mod.pkl', 'wb') as f:
    pickle.dump(traindata, f)

In [7]:
# Load the precessed traindata
with open('train_mod.pkl', 'rb') as f:
    traindata_embeddings = pickle.load(f)

In [22]:
traindata_embeddings

,user_id,parent_asin,rating,timestamp,history,title_review,text,asin,title_meta,author,categories,description,average_rating,price,store,details,item_embedded,genres_embedded,author_tokenized,description_embedded
0,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,048643107X,5.0,1481688109000,NaN,Five Stars,Great resource on history of fair Isle knitti...,048643107X,Traditional Fair Isle Knitting (Dover Knitting...,sheilamcgregor,NaN,Situated far to the north of the Scottish main...,4.7,18.95,Sheila McGregor (Author),{'Publisher': 'Dover Publications (September 1...,[34645],"[-0.06429783, -0.028240208, 0.045258418, -0.03...",[48747],"[-0.04587563, 0.015745105, 0.031058202, -0.072..."
1,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1584799390,5.0,1481688355000,048643107X,Good time.,Gorgeous book. There are a few patterns that I...,1584799390,My Grandmother's Knitting: Family Stories and ...,larissabrown,NaN,When most people think of their grandmother’s ...,4.2,11.99,"Larissa Brown (Author), Michael Crouser (Phot...","{'Publisher': 'Stewart, Tabori and Chang (Sept...",[95931],"[-0.06429783, -0.028240208, 0.045258418, -0.03...",[31395],"[-0.098614596, -0.075296074, -0.038503904, -0...."
2,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1606600478,5.0,1481688478000,048643107X 1584799390,Arrived in great time. Plan to make a few of t...,Gorgeous book. Well packed. Arrived in great t...,1606600478,Tudor Roses,alicestarmore,NaN,"Part fashion, part photography, part English h...",4.7,53.83,Alice Starmore (Author),"{'Publisher': 'Calla Editions; Revised, Expand...",[101582],"[-0.06429783, -0.028240208, 0.045258418, -0.03...",[1527],"[-0.078143656, 0.027341599, 0.0073783463, 0.00..."
3,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1596684372,5.0,1482218160000,048643107X 1584799390 1606600478,Five Stars,Good resource for simple fair isle patterns.,1596684372,200 Fair Isle Motifs: A Knitter's Directory,maryjanemucklestone,NaN,The comprehensive guide to Fair Isle knitting ...,4.8,16.49,Mary Jane Mucklestone (Author),{'Publisher': 'Interweave; Illustrated edition...,[99490],"[-0.06429783, -0.028240208, 0.045258418, -0.03...",[36392],"[-0.04729475, -0.024859317, -0.0013965741, -0...."
4,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,0918804973,5.0,1482218794000,048643107X 1584799390 1606600478 1596684372,Good book.,Alice Starmore is one of my favorite Fair Isle...,0918804973,Alice Starmore's Book of Fair Isle Knitting,alicestarmore,NaN,Scotland's Fair Isle has long been known for i...,4.8,47.7,Alice Starmore (Author),{'Publisher': 'Taunton Press; Reprint edition ...,[63683],"[-0.06429783, -0.028240208, 0.045258418, -0.03...",[1527],"[-0.006313376, -0.024613753, 0.00386671, -0.03..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208939,AGH7KQD653ALUVNARQ7WD6YFJPNA,0997560207,5.0,1466519703000,0814436439,One of the Best Sales Leadership Books from On...,"David Brock is a brilliant sales expert, one o...",0997560207,Sales Manager Survival Guide: Lessons From Sal...,mr.davidabrock,NaN,"Finally! The definitive guide to the toughest,...",4.6,18.95,Mr. David A Brock (Author),{'Publisher': 'Partners In EXCELLENCE; 1st edi...,[67352],"[-0.06429783, -0.028240208, 0.045258418, -0.03...",[],"[-0.06570472, -0.049146745, -0.06108873, -0.04..."
208940,AGH7KQD653ALUVNARQ7WD6YFJPNA,1626343888,5.0,1493383948000,0814436439 0997560207,"like ""fighting fires"" and swooping in as the ""...",The Sales Manager role is arguably one of the ...,1626343888,The Sales Manager's Guide to Greatness: Ten Es...,kevinf.davis,NaN,"2018 Axiom Business Book Award Winner, Silver ...",4.5,24.95,Kevin F. Davis (Author),{'Publisher': 'Greenleaf Book Group Press (Mar...,[105298],"[-0.06429783, -0.028240208, 0.045258418, -0.03...",[37],"[-0.01935307, 0.026034739, -0.004637217, -0.03..."
208941,AFVMMNIFDICHHSD5DOLM46TN4CZA,1483365751,5.0,1455464856000,NaN,A Must Read for Transformative Educators,I've been an educator for nearly 20 years. The...,1483365751,UnCommon Learning: Creating Schools That Work ...,ericc.sheninger,NaN,UnCommon Learn

In [32]:
### Tokenize the items and authors for the traindata 
# Tokenize the items purchased (parent_asin)
traindata['item_embedded'] = Tokenizer_book_id.texts_to_sequences(traindata['parent_asin'])

# Tokenize the authors (clean spaces, lowercase, and handle missing values)
traindata['author'] = [author.replace(' ', '').lower() if pd.notna(author) else 'unk' for author in traindata['author']]

# Tokenize the authors column 
traindata['author_tokenized'] = Tokenizer_author.texts_to_sequences(traindata['author'])
# Merge with the descriptions and genres embedded 
traindata_embeddings_selected = traindata_embeddings[['user_id', 'parent_asin', 'description_embedded', 'genres_embedded']]
traindata = pd.merge(traindata, traindata_embeddings_selected, on = ['user_id','parent_asin'], how = 'left') 


In [33]:
traindata[traindata['user_id'] == 'AF5MJENGAL5G6BUFIVMH5LRSZ6EA']

,user_id,parent_asin,rating,timestamp,history,title_review,text,asin,title_meta,author,categories,description,average_rating,price,store,details,item_embedded,author_tokenized,description_embedded,genres_embedded
0,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,048643107X,5.0,1481688109000,NaN,Five Stars,Great resource on history of fair Isle knitti...,048643107X,Traditional Fair Isle Knitting (Dover Knitting...,sheilamcgregor,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...",Situated far to the north of the Scottish main...,4.7,18.95,Sheila McGregor (Author),{'Publisher': 'Dover Publications (September 1...,[31547],[48747],"[-0.04587563, 0.015745105, 0.031058202, -0.072...","[-0.06429783, -0.028240208, 0.045258418, -0.03..."
1,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1584799390,5.0,1481688355000,048643107X,Good time.,Gorgeous book. There are a few patterns that I...,1584799390,My Grandmother's Knitting: Family Stories and ...,larissabrown,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...",When most people think of their grandmother’s ...,4.2,11.99,"Larissa Brown (Author), Michael Crouser (Phot...","{'Publisher': 'Stewart, Tabori and Chang (Sept...",[85551],[31395],"[-0.098614596, -0.075296074, -0.038503904, -0....","[-0.06429783, -0.028240208, 0.045258418, -0.03..."
2,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1606600478,5.0,1481688478000,048643107X 1584799390,Arrived in great time. Plan to make a few of t...,Gorgeous book. Well packed. Arrived in great t...,1606600478,Tudor Roses,alicestarmore,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...","Part fashion, part photography, part English h...",4.7,53.83,Alice Starmore (Author),"{'Publisher': 'Calla Editions; Revised, Expand...",[90611],[1527],"[-0.078143656, 0.027341599, 0.0073783463, 0.00...","[-0.06429783, -0.028240208, 0.045258418, -0.03..."
3,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1596684372,5.0,1482218160000,048643107X 1584799390 1606600478,Five Stars,Good resource for simple fair isle patterns.,1596684372,200 Fair Isle Motifs: A Knitter's Directory,maryjanemucklestone,"['Books', 'Arts & Photography', 'Graphic Design']",The comprehensive guide to Fair Isle knitting ...,4.8,16.49,Mary Jane Mucklestone (Author),{'Publisher': 'Interweave; Illustrated edition...,[88760],[36392],"[-0.04729475, -0.024859317, -0.0013965741, -0....","[-0.06429783, -0.028240208, 0.045258418, -0.03..."
4,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,0918804973,5.0,1482218794000,048643107X 1584799390 1606600478 1596684372,Good book.,Alice Starmore is one of my favorite Fair Isle...,0918804973,Alice Starmore's Book of Fair Isle Knitting,alicestarmore,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...",Scotland's Fair Isle has long been known for i...,4.8,47.7,Alice Starmore (Author),{'Publisher': 'Taunton Press; Reprint edition ...,[57439],[1527],"[-0.006313376, -0.024613753, 0.00386671, -0.03...","[-0.06429783, -0.028240208, 0.045258418, -0.03..."
5,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,1440309051,5.0,1504193766236,048643107X 1584799390 1606600478 1596684372 09...,I am very pleased with this book,I am very pleased with this book. It is full o...,1440309051,The Complete Watercolorist's Essential Noteboo...,gordonmackenzie,"['Books', 'Arts & Photography', 'Painting']",This special 10th-anniversary collection combi...,4.8,22.23,Gordon MacKenzie (Author),{'Publisher': 'North Light Books; 12.5.2009 ed...,[71567],[18983],"[-0.0077486653, 0.014857503, 0.10468955, -0.04...","[-0.06429783, -0.028240208, 0.045258418, -0.03..."
6,AF5MJENGAL5G6BUFIVMH5LRSZ6EA,0312388098,5.0,1504193900779,048643107X 1584799390 1606600478 1596684372 09...,This was a great used book purchase,This was a great used book purchase. There are...,0312388098,Country Weekend Knits: 25 Classic Patterns for...,madelineweston,"['Books', 'Crafts, Hobbies & Home', 'Crafts & ...","Fashions come and go, but some designs are tru...",4.7,16.66,Madeline Weston (Author),"{'Publisher': ""St. Martin's Griffin; First Edi...",[13147],[34454],"[-0.042680908, 0.012707173, 0.04742914, -0.012...","[-0.06429783, -0.028240208, 0.0

In [34]:
MASK_INDEX = 0 
def randomly_mask(input, training = True):
    """
    If training == True:

      This function will randomly mask one index of the input training

    If training == False:

      This function will mask the last index of the input

    Along with the input (input), two lists masked indices and mask_targt
    will be given to the model to keep track of the masked indices and their target respectively.

    """
    masked_indices = []
    mask_target = []

    # Truncate to the MAX_LENGTH
    input_truncated = input[-(MAX_LENGTH):]

    if training:
        # Check if the history is not empty
        if len(input_truncated) > 0:
            # Randomly choose an index to mask
            mask_index = random.randint(0, len(input_truncated) - 1)
            masked_indices.append(mask_index)

            # Mask the item by replacing it with the MASK_TOKEN
            mask_target.append(input_truncated[mask_index])
            input_truncated[mask_index] = MASK_INDEX #index of the mask

    else:
        mask_target.append(input_truncated[-1])
        input_truncated[-1] = MASK_INDEX

    return input_truncated, masked_indices, mask_target

In [35]:
def pad_tokens(x, Value = 0):
    """
    used for authors, ratings, timestamps or any instances where we are dealing with a list to add padding 
    """
    # Ensure x is a list, convert if not
    if not isinstance(x, list):  # Check if x is not a list
        x = [x]  # Convert to list if not already a list

    # Flatten the list of tokenized authors
    #flat_list = list(itertools.chain(*x))

    # Pad the flattened list to the desired length
    padded_list = pad_sequences([x], maxlen=MAX_LENGTH, padding='post', truncating='post', value=Value , dtype='int64')

    return padded_list
    
# Function to pad or truncate the descriptions to the desired length
def mod_dimensions(input, max_length=MAX_LENGTH, embedding_size=384):
    # Ensure x is a list, convert if not
    if not isinstance(input, list):  # Check if x is not a list
        input = input.tolist()

    # Truncate if the number of descriptions is greater than max_length
    input = input[-max_length:]

    # If the number of descriptions is less than max_length, pad with zero vectors
    current_length = len(input)
    if current_length < max_length:
        padding = [np.zeros(embedding_size)] * (max_length - current_length)
        input.extend(padding)  # Add zero vectors

    return np.array(input)  # Convert to numpy array

In [12]:
import random
MASK_INDEX = 0 
def randomly_mask(input, training=True, mask_prob=0.15, last_token_prob=0.30, mask_index=MASK_INDEX):
    """
    If training == True:
    - This function will randomly mask one index of the input, with a 30% chance of masking only the last token.

    If training == False:
    - This function will mask the last index of the input.

    Parameters:
    - input (list): The input list of tokens to be masked.
    - training (bool): Whether in training mode (True) or evaluation mode (False).
    - mask_prob (float): Probability of masking random tokens.
    - last_token_prob (float): Probability of masking only the last token in the sequence.
    - mask_index (int): The token index used for masking (e.g., `[MASK]` token index).

    Returns:
    - input_truncated (list): The input sequence with masked tokens.
    - masked_indices (list): Indices of the masked tokens.
    - mask_target (list): The original values of the masked tokens.
    """

    masked_indices = []
    mask_target = []

    # Truncate to the MAX_LENGTH
    input_truncated = input[-MAX_LENGTH:]

    if training:
        # Check if the history is not empty
        if len(input_truncated) > 0:
            mask_applied = False  # Flag to track if any mask has been applied

            for i in range(len(input_truncated)):
                # Randomly decide if the token should be masked
                if random.random() < last_token_prob and i == len(input_truncated) - 1 and not mask_applied:
                    # 30% chance to mask the last token
                    masked_indices.append(i)
                    mask_target.append(input_truncated[i])
                    input_truncated[i] = mask_index  # Mask the last token
                    mask_applied = True  # Mark that a mask was applied

                elif random.random() < mask_prob and i != len(input_truncated) - 1 and not mask_applied:
                    # 15% chance to mask a random token (not the last token)
                    masked_indices.append(i)
                    mask_target.append(input_truncated[i])
                    input_truncated[i] = mask_index  # Mask the random token
                    mask_applied = True  # Mark that a mask was applied

            # If no mask was applied, fall back to mask the last token to ensure at least one mask
            if not mask_applied:
                masked_indices.append(len(input_truncated) - 1)
                mask_target.append(input_truncated[-1])
                input_truncated[-1] = mask_index  # Mask the last token

    else:
        # If not training, mask only the last token
        masked_indices.append(len(input_truncated) - 1)
        mask_target.append(input_truncated[-1])
        input_truncated[-1] = mask_index  # Mask the last token

    return input_truncated, masked_indices, mask_target


In [36]:
def prepare_train_data(traindata):
    """
    Prepares the training data by tokenizing, padding, and masking relevant features.

    Parameters:
    - traindata (pd.DataFrame): The raw training data with user-item interactions.
    - Tokenizer_user_id (Tokenizer): A tokenizer instance for user IDs.
    - mod_dimensions (function): A function to modify embedded dimensions (e.g., embeddings).
    - pad_tokens (function): A function to pad tokenized lists.
    - randomly_mask (function): A function to randomly mask tokens in item embeddings.
    - MAX_LENGTH (int): Maximum sequence length for padding.

    Returns:
    - train_input (np.ndarray): Padded item embeddings after masking.
    - mask_indices_train (np.ndarray): Indices of masked items in the training data.
    - train_target (np.ndarray): The book IDs of the masked items.
    - grouped_data (pd.DataFrame): Grouped and processed DataFrame containing the features.
    """
    # Group by user_id and aggregate the data
    grouped_data = traindata.groupby('user_id').agg({
        'author_tokenized': lambda x: list(itertools.chain(*x)),
        'description_embedded': lambda x: mod_dimensions(x),
        'genres_embedded': lambda x: mod_dimensions(x),
        'rating': lambda x: list(x),
        'item_embedded': lambda x: list(itertools.chain(*x)),
        'timestamp': lambda x: list(x),
    }).reset_index()

    # Pad the tokens
    grouped_data['rating'] = grouped_data['rating'].apply(lambda x: pad_tokens(x, 3))
    grouped_data['author_tokenized'] = grouped_data['author_tokenized'].apply(lambda x: pad_tokens(x, 0))
    grouped_data['timestamp'] = grouped_data['timestamp'].apply(lambda x: pad_tokens(x, 0))
    grouped_data['user_id_tokenized'] = grouped_data['user_id'].apply(lambda x: Tokenizer_user_id.texts_to_sequences([x]))

    # Apply random masking to item_embedded
    MASK_INDEX = 0
    output = grouped_data.item_embedded.apply(randomly_mask)

    # Prepare the final train_input (tokenized, masked item embeddings)
    train_input = [item[0] for item in output]
    train_input = pad_sequences(train_input, maxlen=MAX_LENGTH, padding='post', value=0)

    # Extract the masked indices and book IDs (train_target)
    mask_indices_train = np.squeeze(np.array([item[1] for item in output]), axis=1)
    train_target = np.squeeze(np.array([item[2] for item in output]))

    # Mask genres, descriptions, and author for the masked items
    for i, index in enumerate(mask_indices_train):
        # Masking descriptions
        grouped_data.description_embedded.iloc[i][index] = np.zeros((1, 384))
        # Masking genres
        grouped_data.genres_embedded.iloc[i][index] = np.zeros((1, 384))
        # Masking author tokens
        grouped_data.author_tokenized.iloc[i][0][index] = 0

    return train_input, mask_indices_train, train_target, grouped_data


In [37]:
train_input1, mask_indices_train1, train_target1, grouped_data1 = prepare_train_data(traindata)

In [38]:
train_input1[0]

array([     0, 106984, 108774, 115590, 116192, 115355, 116055,      0,
            0,      0,      0,      0], dtype=int32)

In [39]:
mask_indices_train1[0]

0

In [40]:
train_target1[0]

58032

In [41]:
import torch
import numpy as np
from torch.utils.data import TensorDataset

def create_trainset(train_input, mask_indices_train, train_target, grouped_data):
    """Helper function to create TensorDataset."""
    return TensorDataset(
        torch.from_numpy(train_input),  # will be mapped to an embedding layer
        torch.from_numpy(np.squeeze(np.stack(np.array(grouped_data['rating'])), axis=1)),
        torch.from_numpy(np.squeeze(np.stack(np.array(grouped_data['timestamp'])), axis=1)),
        # Book features
        torch.from_numpy(np.squeeze(np.stack(np.array(grouped_data['author_tokenized'])), axis=1)),
        torch.from_numpy(np.stack(np.array(grouped_data['genres_embedded']))),
        torch.from_numpy(np.stack(np.array(grouped_data['description_embedded']))),
        # Masked indices and mask target
        torch.from_numpy(mask_indices_train),
        torch.from_numpy(train_target),
        # Tokenized user ids:
        torch.from_numpy(np.squeeze(np.squeeze(np.stack(grouped_data['user_id_tokenized']), axis=1), axis=1))
    )

# Prepare data for multiple datasets
train_input1, mask_indices_train1, train_target1, grouped_data1 = prepare_train_data(traindata)
train_input2, mask_indices_train2, train_target2, grouped_data2 = prepare_train_data(traindata)
train_input3, mask_indices_train3, train_target3, grouped_data3 = prepare_train_data(traindata)

# Create the datasets using the helper function
trainset1 = create_trainset(train_input1, mask_indices_train1, train_target1, grouped_data1)
trainset2 = create_trainset(train_input2, mask_indices_train2, train_target2, grouped_data2)
trainset3 = create_trainset(train_input3, mask_indices_train3, train_target3, grouped_data3)
# Combine the three datasets 
trainset = ConcatDataset([trainset1, trainset2, trainset3])

In [42]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, ConcatDataset
from tqdm import tqdm  # Import tqdm for progress bars


def remove_duplicates_across_datasets(*datasets):
    """Concatenate datasets, check for duplicates based on the `item_embedded` column, and remove duplicates."""
    
    # Extract tensors from each dataset and track progress
    all_tensors = []
    
    for dataset in tqdm(datasets, desc="Extracting tensors from datasets"):
        # Extract tensors from the dataset and append them to the list
        all_tensors.append([tensor.numpy() for tensor in dataset.tensors])

    # Concatenate all the tensors of `item_embedded` (the first tensor in each dataset)
    item_embedded = np.concatenate([tensors[0] for tensors in all_tensors], axis=0)
    
    # Show progress for finding unique `item_embedded`
    print("Finding unique item_embedded values...")
    
    # Find unique item_embedded entries and their indices
    _, unique_indices = np.unique(item_embedded, axis=0, return_index=True)
    
    # Use tqdm to track progress of creating a new dataset
    print("Creating dataset with unique entries...")
    
    # Create a new dataset with only the unique `item_embedded` rows
    unique_dataset_tensors = []
    
    # Iterate through each tensor in all datasets
    for i in range(len(all_tensors[0])):  # Iterate over each tensor (features like item_embedded, ratings, etc.)
        # Concatenate the relevant tensors from all datasets
        concatenated_tensor = np.concatenate([tensors[i] for tensors in all_tensors], axis=0)
        
        # Select the unique entries based on the unique indices
        unique_tensor = concatenated_tensor[unique_indices]
        
        # Convert back to torch tensor and append to the list
        unique_dataset_tensors.append(torch.tensor(unique_tensor))
    
    # Return the new TensorDataset containing only the unique entries
    unique_dataset = TensorDataset(*unique_dataset_tensors)

    return unique_dataset

# Concatenate and remove duplicates across the three datasets
print("Removing duplicates across datasets...")
trainset = remove_duplicates_across_datasets(trainset1, trainset2, trainset3)

print("Data preparation complete!")


Removing duplicates across datasets...


Extracting tensors from datasets: 100%|██████████| 3/3 [00:00<00:00, 790.93it/s]

Finding unique item_embedded values...
Creating dataset with unique entries...


Data preparation complete!


In [45]:
# Process the validation dataset 
# Group the traindata which is the history data for each user 
grouped_valdata = traindata.groupby('user_id').agg({
    'author_tokenized': lambda x: list(itertools.chain(*x)),
    'description_embedded': lambda x: list(x),
    'genres_embedded': lambda x: list(x),
    'rating': lambda x: list(x),
    'item_embedded': lambda x: list(itertools.chain(*x)),
    'timestamp' : lambda x: list(x),
}).reset_index()
# Drop doplicates in validatino dataset if any
valdata = valdata.drop_duplicates(subset=['user_id'])
# Merge the data we have in validation dataset on each customer id with the traindata
valdata = valdata.merge(grouped_valdata, on='user_id', how='inner')
#valdata

In [47]:
# Defining the validation targets 
val_target = np.squeeze([item for item in Tokenizer_book_id.texts_to_sequences(valdata['parent_asin'].values) if len(item)>0],axis = 1)
valdata['val_target'] = valdata['parent_asin'].apply(lambda x: Tokenizer_book_id.texts_to_sequences([x])[0])

In [50]:
# Find all the unique book_ids in the training dataset 
trainset_unique_ids = set(set(train_target1).union(set(train_target2)).union(set(train_target3)))
unique_items_val = set(val_target)                                             
common_ids = trainset_unique_ids.intersection(unique_items_val) # common book ids that are both in training and validation 

#Get the count of common book IDs
num_common_ids = len(common_ids)

print("Number of common book IDs:", num_common_ids)
ratio = num_common_ids / len(unique_items_val)
print(f"Ratio of all validation parent ASINs to common book IDs: {ratio:.3f}")


Number of common book IDs: 4282
Ratio of all validation parent ASINs to common book IDs: 0.497


In [51]:
# Filter the valdata to only parent asins already in the training set 
valdata  = valdata[valdata['val_target'].apply(lambda x: x[0] in common_ids if len(x)>0 else False)]
valdata

,user_id,parent_asin,rating_x,timestamp_x,history,title_review,text,asin,title_meta,author,...,price,store,details,author_tokenized,description_embedded,genres_embedded,rating_y,item_embedded,timestamp_y,val_target
3,AFG6YQ3GOY7TVFKQ3SKDVS6Q6RDQ,B08CV9SPDQ,4.0,1635609140286,B005LVLZEI 1621470857 B00BIOG1ZU B000FCKBPM B0...,a sweet read,Loved the back and forth future /past theme<br...,B08CV9SPDQ,The Venice Sketchbook: A Novel,Rhys Bowen,...,0.0,Rhys Bowen (Author) Format: Kindle Edition,{'Publisher': 'Lake Union Publishing (April 13...,"[34970, 5597, 47382, 45166]","[[0.016344022, -0.0334038, 0.044654988, 0.0947...","[[-0.06429783, -0.028240208, 0.045258418, -0.0...","[4.0, 5.0, 4.0, 4.0, 5.0]","[109401, 93397, 112184, 105688, 115869]","[1334589222000, 1389290398000, 1394393440000, ...",[124931]
5,AGQFGXUGFSZHNRQGP7J24RVSLZSA,0545480280,5.0,1431563558000,B00AL2V30S B004TO5THM B003IWOC1U B000FCKIGE B0...,very happy with this purchase,grandson loves all the superheroes. this was ...,0545480280,Save the Day (LEGO DC Superheroes: Comic Reader),Trey King,...,13.99,"Trey King (Author), Kenny Kiernan (Illustrator)",{'Publisher': 'Scholastic Inc.; Illustrated ed...,"[49534, 8020, 16805, 22326, 24041, 19564, 4593...","[[0.014230283, -0.025171949, 0.05049709, -0.06...","[[-0.06429783, -0.028240208, 0.045258418, -0.0...","[5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[111752, 108571, 107412, 105700, 111681, 69905...","[1375843592000, 1378483723000, 1379724867000, ...",[34280]
8,AFSLJXRMFXVP3IR4ZIXQERRZO3XA,0071463380,5.0,1533200700632,0689829523 0762106492 1451681755 0764540769 05...,Seems to be a thorough workbook on Spanich.,I had Spanish in high school Lost my old work...,0071463380,Easy Spanish Step-By-Step,Barbara Bregstein,...,10.98,Barbara Bregstein (Author),{'Publisher': 'McGraw Hill; 1st edition (Decem...,"[44016, 34213, 15186, 36971, 53247, 53247]","[[-0.009896521, 0.014383958, 0.05096839, 0.014...","[[-0.06429783, -0.028240208, 0.045258418, -0.0...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[40556, 45395, 73414, 46629, 32761, 6008, 3109...","[1393121505000, 1399743592000, 1412632693000, ...",[6340]
11,AGDR3CBX2GWY3JKGDLMUERVFWPTQ,0471711853,4.0,1456142797000,0470125128 1563924307 1580170986,Four Stars,Great read.,0471711853,Find the Right Mutual Fund: Morningstar Mutual...,Christine Benz,...,27.57,Christine Benz (Author),{'Publisher': 'Wiley; 2nd edition (December 29...,"[34533, 37006]","[[-0.0011565166, -0.0952521, -0.023431309, 0.0...","[[-0.06429783, -0.028240208, 0.045258418, -0.0...","[4.0, 4.0, 4.0]","[30236, 82743, 84801]","[1444427566000, 1444427609000, 1444427975000]",[30862]
13,AGNCQ6VKJ4X24XFV2X6KNGPPMVSQ,0679743464,5.0,1422187242000,0151003084 0393088863 1932234233 0099481553 19...,Five Stars,My first Murakami book. This is a really inter...,0679743464,Hard-Boiled Wonderland and the End of the Worl...,Haruki Murakami,...,NaN,Haruki Murakami (Author),"{'Publisher': 'Vintage (March 2, 1993)', 'Lang...","[14976, 35186, 30799, 19836, 21928, 55741, 307...","[[0.011052213, -0.006400329, -0.026590139, -0....","[[-0.06429783, -0.028240208, 0.045258418, -0.0...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[8663, 22336, 100552, 6752, 101990, 13837, 100...","[1321067188000, 1395984159000, 1395984426000, ...",[39583]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19957,AGNCIY6WXOPBS7WBMSJYKZR2FR7Q,0323059104,5.0,1378252430000,0803623631 1556429762 0803623909,Love it,Even though this book did not come in sealed i...,0323059104,Pediatric Skills for Occupational Therapy Assi...,NaN,...,NaN,"Jean W. Solomon MHS OTR/L FAOTA (Author), J...","{'Publisher': 'Mosby; 3rd edition (January 3, ...","[34170, 34905]","[[-0.00035130788, -0.01735859, 0.010600182, -0...","[[-0.06429783, -0.028240208, 0.045258418, -0.0...","[5.0, 5.0, 5.0]","[51240, 82010, 51241]","[1378252171000, 1378252299000, 1378252356000]",[16376]
19987,AHFJOUTTNZFK4LMNW6PJ4ZIIOJIA,B01MYTCG7O,4.0,13792113810

In [52]:
### Preprocess the Validation dataset

# Tokenize the user ids
valdata['user_id_tokenized'] = valdata['user_id'].apply(lambda x: Tokenizer_user_id.texts_to_sequences([x]))

# Add a zero index for the prediction (Acts like a masked token)
valdata['item_embedded'] = valdata['item_embedded'].apply(lambda x: x + [0] if len(x)<= 11 else x[-11:]+[0]) # truncate if length>12
mask_indices_val = np.array(valdata['item_embedded'].apply(lambda x: len(x)-1)) #if len(x)<12 else 12)) # find the index of the mask
valdata['item_embedded'] = valdata['item_embedded'].apply(lambda x: pad_tokens(x, 0)) # finally pad the items

# Adding a index of zero for the author (UNK author) and then padding the sequences
valdata['author_tokenized'] = valdata['author_tokenized'].apply(lambda x: x[-11:] + [0]) # needs padding
valdata['author_tokenized'] = valdata['author_tokenized'].apply(lambda x: pad_sequences([x], maxlen = 12, padding = 'post', value = 0)) # needs padding

# Adding an array of zero for the description of final item and adding padding for the remaining indices to reach maxlen
valdata['description_embedded'] = valdata['description_embedded'].apply(lambda x: np.concatenate([x[-11:], np.zeros((1,384))])) #needs padding
valdata['description_embedded'] = valdata['description_embedded'].apply(lambda x: mod_dimensions(x))

# Adding an array of zero for the genre of final item and adding padding for the remaining indices to reach maxlen
valdata['genres_embedded'] = valdata['genres_embedded'].apply(lambda x: np.concatenate([x[-11:], np.zeros((1,384))])) #needs padding
valdata['genres_embedded'] = valdata['genres_embedded'].apply(lambda x: mod_dimensions(x))


# Truncate the ratings and add the rating of the target item to the list
valdata['rating_y'] = valdata['rating_y'].apply(lambda x: x[-11:])
valdata['rating_y'] = valdata.apply(
    lambda row: np.concatenate([np.array(row['rating_y']).reshape(-1), np.array([row['rating_x']]).reshape(-1)]),
    axis=1
)
# Add padding to the rating list
valdata['rating_y'] = valdata['rating_y'].apply(lambda x: pad_sequences([x], maxlen = MAX_LENGTH, value = 3, padding = 'post'))

# Truncate the timestamps and add the timestamp of the target item to the list
valdata['timestamp_y'] = valdata['timestamp_y'].apply(lambda x: x[-11:])
valdata['timestamp_y'] = valdata.apply(
    lambda row: np.concatenate([np.array(row['timestamp_y']).reshape(-1), np.array([row['timestamp_x']]).reshape(-1)]),
    axis=1
)
# Add padding to the rating list
valdata['timestamp_y'] = valdata['timestamp_y'].apply(lambda x: pad_sequences([x], maxlen = MAX_LENGTH, value = 0, padding = 'post', dtype = 'int64'))

# Store the target validation values
#val_target = (np.array(Tokenizer_book_id.texts_to_sequences(valdata['parent_asin'].values)))
#valdata['val_target'] = (valdata['parent_asin'].apply(lambda x: Tokenizer_book_id.texts_to_sequences([x])))

/var/folders/ry/1l9br0wd433fd3xsqyszbnlr0000gn/T/ipykernel_1557/3458531857.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valdata['user_id_tokenized'] = valdata['user_id'].apply(lambda x: Tokenizer_user_id.texts_to_sequences([x]))
/var/folders/ry/1l9br0wd433fd3xsqyszbnlr0000gn/T/ipykernel_1557/3458531857.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valdata['item_embedded'] = valdata['item_embedded'].apply(lambda x: x + [0] if len(x)<= 11 else x[-11:]+[0]) # truncate if length>12
/var/folders/ry/

In [53]:
valdata

,user_id,parent_asin,rating_x,timestamp_x,history,title_review,text,asin,title_meta,author,...,store,details,author_tokenized,description_embedded,genres_embedded,rating_y,item_embedded,timestamp_y,val_target,user_id_tokenized
3,AFG6YQ3GOY7TVFKQ3SKDVS6Q6RDQ,B08CV9SPDQ,4.0,1635609140286,B005LVLZEI 1621470857 B00BIOG1ZU B000FCKBPM B0...,a sweet read,Loved the back and forth future /past theme<br...,B08CV9SPDQ,The Venice Sketchbook: A Novel,Rhys Bowen,...,Rhys Bowen (Author) Format: Kindle Edition,{'Publisher': 'Lake Union Publishing (April 13...,"[[34970, 5597, 47382, 45166, 0, 0, 0, 0, 0, 0,...","[[0.016344022005796432, -0.033403798937797546,...","[[-0.06429783254861832, -0.028240207582712173,...","[[4, 5, 4, 4, 5, 4, 3, 3, 3, 3, 3, 3]]","[[109401, 93397, 112184, 105688, 115869, 0, 0,...","[[1334589222000, 1389290398000, 1394393440000,...",[124931],[[4]]
5,AGQFGXUGFSZHNRQGP7J24RVSLZSA,0545480280,5.0,1431563558000,B00AL2V30S B004TO5THM B003IWOC1U B000FCKIGE B0...,very happy with this purchase,grandson loves all the superheroes. this was ...,0545480280,Save the Day (LEGO DC Superheroes: Comic Reader),Trey King,...,"Trey King (Author), Kenny Kiernan (Illustrator)",{'Publisher': 'Scholastic Inc.; Illustrated ed...,"[[17492, 20243, 37183, 47992, 38661, 38661, 38...","[[-0.05019771307706833, -0.06004162132740021, ...","[[-0.06429783254861832, -0.028240207582712173,...","[[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]","[[89101, 84871, 87366, 13068, 12910, 13666, 12...","[[1426552820000, 1426552850000, 1426554822000,...",[34280],[[6]]
8,AFSLJXRMFXVP3IR4ZIXQERRZO3XA,0071463380,5.0,1533200700632,0689829523 0762106492 1451681755 0764540769 05...,Seems to be a thorough workbook on Spanich.,I had Spanish in high school Lost my old work...,0071463380,Easy Spanish Step-By-Step,Barbara Bregstein,...,Barbara Bregstein (Author),{'Publisher': 'McGraw Hill; 1st edition (Decem...,"[[44016, 34213, 15186, 36971, 53247, 53247, 0,...","[[-0.009896521456539631, 0.014383957721292973,...","[[-0.06429783254861832, -0.028240207582712173,...","[[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]","[[40556, 45395, 73414, 46629, 32761, 6008, 310...","[[1393121505000, 1399743592000, 1412632693000,...",[6340],[[9]]
11,AGDR3CBX2GWY3JKGDLMUERVFWPTQ,0471711853,4.0,1456142797000,0470125128 1563924307 1580170986,Four Stars,Great read.,0471711853,Find the Right Mutual Fund: Morningstar Mutual...,Christine Benz,...,Christine Benz (Author),{'Publisher': 'Wiley; 2nd edition (December 29...,"[[34533, 37006, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]","[[-0.0011565166059881449, -0.09525209665298462...","[[-0.06429783254861832, -0.028240207582712173,...","[[4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3]]","[[30236, 82743, 84801, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1444427566000, 1444427609000, 1444427975000,...",[30862],[[12]]
13,AGNCQ6VKJ4X24XFV2X6KNGPPMVSQ,0679743464,5.0,1422187242000,0151003084 0393088863 1932234233 0099481553 19...,Five Stars,My first Murakami book. This is a really inter...,0679743464,Hard-Boiled Wonderland and the End of the Worl...,Haruki Murakami,...,Haruki Murakami (Author),"{'Publisher': 'Vintage (March 2, 1993)', 'Lang...","[[19836, 19836, 19836, 19836, 19836, 19836, 19...","[[-0.010434385389089584, 0.08200400322675705, ...","[[-0.06429783254861832, -0.028240207582712173,...","[[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]]","[[39621, 19667, 39646, 10652, 39705, 19620, 10...","[[1422186982000, 1422186986000, 1422186989000,...",[39583],[[14]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19957,AGNCIY6WXOPBS7WBMSJYKZR2FR7Q,0323059104,5.0,1378252430000,0803623631 1556429762 0803623909,Love it,Even though this book did not come in sealed i...,0323059104,Pediatric Skills for Occupational Therapy Assi...,NaN,...,"Jean W. Solomon MHS OTR/L FAOTA (Author), J...","{'Publisher': 'Mosby; 3rd edition (January 3, ...","[[34170, 34905, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]","[[-0.00035130788455717266, -0.0173585899174213...","[[-0.06429783254861832, -0.028240207582712173,...","[[5, 5

In [54]:
valset = TensorDataset(torch.from_numpy(np.squeeze(np.stack(valdata['item_embedded']),axis = 1)), #will be mapped to an embedding layer
                         torch.from_numpy(np.squeeze(np.stack(valdata['rating_y']),axis = 1)),
                         torch.from_numpy(np.squeeze(np.stack(valdata['timestamp_y']),axis = 1)),
                         # Book features
                         torch.from_numpy(np.squeeze(np.stack(valdata['author_tokenized'].values),axis = 1 )), # good to go
                         torch.from_numpy(np.stack((valdata['genres_embedded']))), #good to go
                         torch.from_numpy(np.stack(valdata['description_embedded'])),# good to go
                         # Masked indices and mask target
                         torch.from_numpy(mask_indices_val), # check to make sure the dims are compatible
                         torch.from_numpy(np.squeeze(np.stack(valdata.val_target),axis = 1)),
                         # Customer ids tokenized
                         torch.from_numpy(np.squeeze(np.squeeze(np.stack(valdata.user_id_tokenized),axis = 1),axis = 1)))

### The train dataset is fully defined as:
 - **train_input**: which is the masked book_ids in the purchase history
 - **ratings** given to each item purchased by the customer --> will be mapped to an embedding and added to the item embedding
 - **timestamp** for each purchase: will be normalized and shift the attention weights in the mha
 - **the author tokenized** is a vector of 12 indices. for the not given authors, we replaced 0.
 - **genres** of each book are grouped into a single string and then embedded using a sentence tokenizer. --> can be mapped to the desired dim
 - **descriptions** for each book are embedded using the sentence tokenizer
 - **mask_indices_train**: this is the index of the masked book_id --> will be used to extract the corresponding representation from the output
 - **train_target**: will be used to check the accuracy of the model in making the prediction.

### Plan for the book features:
The features of the book will be added to the book_item embedding to provide a richer representation of each book.



#### Possible Modifications:
- Add another embedding layer for each customer id. Then when we are doing valdata given the customer id, we have an embedding which encodes the past behaviour of this customer.

- what if we take the 20 most probable book_items and check if the 20 most probable have the item purchased in them.
- add the ratings before vs after the bert representations also add them to the attention matrices to check.
#### Questions
? the author/genre/description/rating of the masked item must also be masked to make sure the model has no information about the book masked.
but not the rating and the timestamp.

 Do you wanna try the embedding track of the book_ids, ratings and the customer ids? customer ids are important because them we will hopefully capture info about
#each customer and this info can be used in the valdataset.

We will use two approaches: one we will concatenate for a totol size of 256 and another we will add for a total size of 256.

In [55]:
# Postitional Encoding Layer
class Pos_Enc(nn.Module):

    def __init__(self, dropout_rate = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p = dropout_rate)


    def get_angles(self,len_seq, d_model):

        """
        Input
        x  : input samples with shape (#samples, len_seq, len_emb = d_model)

        """
        # Initialize the parameters
        angles = torch.zeros((len_seq, d_model // 2))

        for pos in range(len_seq):
            for i in range(d_model//2):
                angles[pos,i] = pos/(10000**(2*i/d_model))
        return(angles)

    def forward(self,x):
        """
        This function will calculate the positional encodings for a given input x
        Input
        x   : input sequences with shape (#samples, len_seq, len_emb)

        Output
        pos_encoding (tensor): denoting the position of words in the sequence; shape = (1, len_seq, len_emb)

        """

        # Initialize variables
        len_seq = x.shape[1]
        d_model = x.shape[2]
        pos_encoding = torch.zeros((1,len_seq, d_model))
        # Calculate the angles
        angles = self.get_angles(len_seq,d_model)
        # we would need a tensor of 1, len_seq, d_model) first for loop is len_seq
        for pos in range(len_seq):
            for i in range(angles.shape[1]):
                pos_encoding[:,pos, 0::2] = torch.sin(angles[pos,:])
                pos_encoding[:,pos, 1::2] = torch.cos(angles[pos,:])

        # register pos_encoding as a buffer in the modul
        pos_encoding.requires_grad = False # not trainable

        # Add the positional Encodings to the input
        x = x + pos_encoding

        # Apply Dropout and return
        return(self.dropout(x))


In [56]:
class RFeedForward(nn.Module):

    def __init__(self, d_model,d_ff,p):
        super().__init__()
        self.d_model = d_model
        self.layer1 = nn.Linear(d_model, d_ff)
        self.layer2 = nn.Linear(d_ff,d_model)
        self.relu   = nn.ReLU()
    def forward(self, x):

        """
        This class defines the feed forward layer of the transformer
        - A dropout layer will be applied between the two neural layers
        """

        return self.layer2(self.relu(self.layer1(x))) # shape: (#samples, len_seq, d_model)


In [57]:
# Define a feedforward layer using GELU activation
class GFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, p=0.1):
        super(GFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.gelu = nn.GELU()  # Using GELU instead of ReLU
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = self.linear1(x)
        x = self.gelu(x)  # Apply GELU activation
        x = self.dropout(x)
        x = self.linear2(x)
        return x


In [58]:
class TMultiHeadAttn(nn.Module):

    def __init__(self,num_heads, d_model, decay_rate = 0.1):
        super(TMultiHeadAttn,self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_params = d_model//num_heads
        self.decay_rate = decay_rate

        self.W_q = nn.Linear(d_model, d_model) # mapping to the same number of features
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        # Check if the number of len_emb (d_model) is divisable by num_heads
        assert d_model % num_heads == 0, "d_model is not divisable by the number of heads"


    def self_attention(self, q, k, v, timestamps, mask):

        # Calculate the dot produce of Q and K
        dotqk = torch.matmul(q, k.transpose(-2,-1))/(self.head_params**0.5) # shape = (#samples, num_heads, len_seq, len_seq)
        # Apply the mask if givne
        if mask is not None:
            dotqk = dotqk.masked_fill(mask == 0, -1e9)
        # # Compute time difference matrix (NxN)
        time_diff = timestamps.unsqueeze(2) - timestamps.unsqueeze(1)
        decay = (torch.exp(-self.decay_rate * time_diff.abs())).unsqueeze(1) # Apply an exponential decay function
        # Rescale the attention weights
        adjusted_attention = dotqk * decay
        # Apply the Softmax to the attention weights
        attention_scores = torch.softmax(adjusted_attention, dim = -1)
        # Multiply to the value matrix
        result = torch.matmul(attention_scores.float(), v)
        return(attention_scores,result)

    def split_heads(self, q):

        """
        Reshaping
        Input of shape : (#samples, seq_len, d_model)

        to
        output of shape: (#samples, num_heads, seq_len, head_parameters)
        """
        samples, seq_len, _ = q.shape
        return q.view(samples, seq_len, self.num_heads, self.head_params).transpose(1,2)

    def forward(self, q,k,v,timestamps, mask=None):

        # Define Query, Key, Value
        Query = self.split_heads(self.W_q(q))
        Key  = self.split_heads(self.W_k(k))
        Value = self.split_heads(self.W_v(v))
        # Run the self-attention
        attention_scores, attn_output = self.self_attention(Query, Key,Value, timestamps, mask)

        # Concatenate the heads
        batch_size, _,seq_len, _= attn_output.shape
        attn_output = attn_output.transpose(1,2).contiguous().view(batch_size, seq_len, self.d_model) #shape = (#samples, len_seq, d_model)

        # Apply through the last linear layer
        result_final = self.W_o(attn_output) #shape = (#samples, len_seq, d_model)

        return result_final


In [297]:
from torch.utils.data import Subset, DataLoader

# Choose the indices of the samples you want to keep (e.g., first 1000 samples)
indices = list(range(10))  # You can choose any indices here

# Create a subset of the dataset
subset_trainset = Subset(trainset, indices)

# Create a DataLoader for the subset
subset_trainloader = DataLoader(subset_trainset, batch_size=10, shuffle=False)


In [31]:
# Define to normalize the time stamps 
def normalize_time(timestamps):
    """
    This function will normalize and create the recency parameter.
    """
    timestamps = timestamps.clone().detach().float()
    max_ts = timestamps.max(dim=1, keepdim=True)[0]  # max across each row (user's history)
    normalized_timestamps = timestamps / max_ts
    return normalized_timestamps

In [238]:
def split_heads(q):

    """
    Reshaping
    Input of shape : (#samples, seq_len, d_model)

    to
    output of shape: (#samples, num_heads, seq_len, head_parameters)
    """
    samples, seq_len, _ = q.shape
    return q.view(samples, seq_len, 2, 5).transpose(1,2)


In [226]:
d_model = 10
W_q = nn.Linear(d_model, d_model) # mapping to the same number of features
W_k = nn.Linear(d_model, d_model)
W_v = nn.Linear(d_model, d_model)
W_o = nn.Linear(d_model, d_model)

In [320]:
for batch in subset_trainloader: 
    train_input = batch[0]
    q = nn.Embedding(len(Tokenizer_book_id.word_index)+10, 10)(train_input) 
    ratings = batch[1]
    timestamps = batch[2]
    authors = batch[3]
    genre = batch[4]
    description = batch[5]
    user_ids = batch[8]
    mask_indices = batch[6]  # Indices to be masked
    train_target = batch[7]
    timestamps_normed = normalize_time(timestamps) 
    Query = split_heads(W_q(q))
    Key = split_heads(W_k(q))
    Value = split_heads(W_v(q))
    dotqk = torch.matmul(Query, Key.transpose(-2,-1))/(5**0.5) # shape = (#samples, num_heads, len_seq, len_seq)
    mask = (train_input != 0).unsqueeze(1).unsqueeze(2)
    dotqk = dotqk.masked_fill(mask == 0, -1e9)
    time_diff = timestamps_normed.unsqueeze(2) - timestamps_normed.unsqueeze(1)
    decay = (torch.exp(-(0.1) * time_diff.abs())).unsqueeze(1)
    
    adjusted_attention = dotqk * decay
    attention_scores = torch.softmax(adjusted_attention, dim = -1)
    attention_scores_without_decay  = torch.softmax(dotqk, dim = -1)

    result = torch.matmul(attention_scores.float(), Value)
    # Concatenate the heads
    batch_size, _,seq_len, _= result.shape
    attn_output = result.transpose(1,2).contiguous().view(batch_size, seq_len, d_model) #shape = (#samples, len_seq, d_model)

    # Apply through the last linear layer
    result_final = W_o(attn_output)

In [319]:
timestamps.shape

torch.Size([10, 12])

In [331]:
mask_indices[0:3]

tensor([3, 2, 2])

In [340]:
ratings[0:3]

tensor([[5, 5, 5, 5, 4, 4, 5, 3, 3, 3, 3, 3],
        [5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [2, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3]])

In [338]:
rating_of_masked_items = ratings[torch.arange(ratings.shape[0]), mask_indices]


In [339]:
rating_of_masked_items

tensor([5, 3, 5, 5, 3, 5, 5, 5, 5, 5])

In [59]:
# Custom encoder class for attention and feed-forward layers
class EncoderLayer(nn.Module):
    def __init__(self, config):
        super(EncoderLayer, self).__init__()

        self.attn = TMultiHeadAttn(config.num_attention_heads, config.hidden_size)  # Multi-head attention
        self.norm1 = nn.LayerNorm(config.hidden_size)  # Layer normalization
        self.ffn = GFeedForward(config.hidden_size, config.intermediate_size, config.hidden_dropout_prob)  # Feed-forward
        self.dropout = nn.Dropout(config.hidden_dropout_prob)  # Dropout

    def forward(self, x, timestamps, attention_mask):
        # Multi-head attention
        attn_output = self.attn(x, x, x, timestamps, attention_mask)
        attn_output = self.dropout(attn_output)

        # Add the residual connection and normalize
        x = self.norm1(attn_output + x)

        # Feed-forward network
        ffn_output = self.ffn(x)
        ffn_output = self.dropout(ffn_output)

        return self.norm1(ffn_output + x)  # Another residual connection with normalization


In [60]:
class CustomBert(nn.Module):
    def __init__(self, config):
        super(CustomBert, self).__init__()

        self.len_emb = config.hidden_size
        self.decay_rate = 0.1
        self.dropout = nn.Dropout(p=config.hidden_dropout_prob)  # Dropout layer
        self.norm1 = nn.LayerNorm(config.hidden_size)  # Layer Normalization
        self.norm2 = nn.LayerNorm(config.hidden_size)  # Layer Normalization
        self.norm3 = nn.LayerNorm(config.hidden_size)  # Layer Normalization
        self.norm4 = nn.LayerNorm(config.hidden_size)
        self.norm5 = nn.LayerNorm(config.hidden_size)
        self.norm6 = nn.LayerNorm(config.hidden_size)

        # Embedding layers for different features like ratings, train_input, and authors
        self.embedding_layers = nn.ModuleList([nn.Embedding(size, config.hidden_size) for size in config.sizes])

        # Linear layers for genre and description features
        self.linear_layers = nn.ModuleList([nn.Linear(384, config.hidden_size) for _ in range(2)])

        # Embedding layer for user_id
        #self.embed_user = nn.Embedding(len(Tokenizer_user_id.word_index), config.hidden_size)
        # Embedding of rating
        self.embed_ratings = nn.Embedding(6, config.hidden_size) 

        # Positional encoding and dropout
        self.pos_enc = Pos_Enc(dropout_rate = config.hidden_dropout_prob)

        # Encoder to hold multiple layers
        self.encoder_layers = nn.ModuleList([EncoderLayer(config) for _ in range(config.num_enc_layers)])

        # Output layer
        self.linear = nn.Linear(config.hidden_size, config.item_size)

    def normalize_time(self, timestamps):
        """
        This function will normalize and create the recency parameter.
        """
        timestamps = timestamps.clone().detach().float()
        max_ts = timestamps.max(dim=1, keepdim=True)[0]  # max across each row (user's history)
        normalized_timestamps = timestamps / max_ts
        return normalized_timestamps

    def forward(self, input):
  
        """
        Input will be loaded by the dataloader and will be a tuple of values.
        """
        train_input = input[0]
        ratings = input[1] 
        timestamps = input[2]
        authors = input[3] 
        genre = input[4]#(samples, max_len, 384) 
        description = input[5] #(samples, max_len, 384) 
        #user_ids = input[8]
        mask_indices = input[6]
        rating_of_masked_items = ratings[torch.arange(ratings.shape[0]), mask_indices]

        # Embedding layer for train_input, authors, and ratings with normalization
        combined_embed = sum([
            self.norm1(self.embedding_layers[i](x))
            for i, x in enumerate([train_input, authors])
        ])

        # Add the genre and description layers with dropout and normalization
        combined_embed += sum([
            self.norm2(self.linear_layers[i](x.float()))
            for i, x in enumerate([genre, description])
        ])

        # Map the user id to an embedding layer and add to the combined embedding with dropout and normalization
        #user_embedded = self.norm6(self.embed_user(user_ids))
        #user_embedded = user_embedded.unsqueeze(1).repeat(1, combined_embed.size(1), 1)  # (batch_size, len_seq, len_emb)
        #combined_embed += user_embedded  # combining user_id, book_ids, and book features

        # Add positional encoding and normalize
        combined_embed = self.pos_enc(combined_embed) * torch.sqrt(torch.tensor(self.len_emb)) #there is a dropout layer added within pos_enc
        

        # Define attention mask (1 if not zero, 0 if zero)
        attention_mask = (train_input != 0).unsqueeze(1).unsqueeze(2)  # size = (batch_size, 1, 1, len_seq)

        # Normalize the timestamps
        timestamp_norm = self.normalize_time(timestamps)

        # Pass through the encoder layers 
        encoder_output = combined_embed
        for encoder_layer in self.encoder_layers:
            # Pass through the encoder layer
            encoder_output = encoder_layer(encoder_output, timestamp_norm, attention_mask)

            # Apply dropout after each encoder layer's output
            encoder_output = self.dropout(encoder_output)

            # Apply layer normalization
            encoder_output = self.norm3(encoder_output)
            
        # Extract the encoder output of the masked indices
        encoder_masked_items = encoder_output[torch.arange(encoder_output.shape[0]), mask_indices]
        # Map the ratings of masked items and add to the BERT representation
        encoder_masked_items += self.embed_ratings(rating_of_masked_items)
        # Final output layer
        final_output = self.linear(self.norm4(self.dropout(encoder_masked_items)))

        return final_output


alternatively, we can make prediction using the following: 
# Extract the last timestamp and last rating for each sequence in the batch
        last_timestamp = timestamps[:, -1].unsqueeze(-1)  # Shape: (batch_size, 1)
        last_rating = ratings[:, -1].unsqueeze(-1)  # Shape: (batch_size, 1)

        # Normalize the last timestamp (if needed)
        last_timestamp_norm = self.normalize_time(last_timestamp)

        # Concatenate the last timestamp, last rating, and encoder output
        last_info = torch.cat([last_timestamp_norm, last_rating.unsqueeze(-1)], dim=-1)  # Shape: (batch_size, 2)
        encoder_output_with_last_info = torch.cat([encoder_output, last_info.unsqueeze(1).repeat(1, encoder_output.size(1), 1)], dim=-1)

        # Final output layer to make the prediction
        final_output = self.linear(encoder_output_with_last_info)

        return final_output

In [61]:
len_book_ids = len(unique_book_ids)
len_authors = len(Tokenizer_author.word_index)
len_ratings = 6
len_user_ids = len(Tokenizer_user_id.word_index)
sizes = [len_book_ids+10, len_authors+10]#len_user_ids+10, len_ratings]
sizes

[128740, 55926]

In [62]:
class CustomBertConfig:
    def __init__(self, hidden_size=64, num_attention_heads=16, num_hidden_layers=12,
                 intermediate_size=64, hidden_dropout_prob=0.3,
                 attention_probs_dropout_prob=0.1, max_position_embeddings=512,
                 vocab_size=len(Tokenizer_book_id.word_index), type_vocab_size=2, initializer_range=0.02,
                 layer_norm_eps=1e-12):
        self.hidden_size = hidden_size
        self.num_attention_heads = num_attention_heads
        self.num_enc_layers = NUM_ENC_LAYERS
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = hidden_dropout_prob
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.item_size = len(Tokenizer_book_id.word_index)
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range
        self.layer_norm_eps = layer_norm_eps
        self.sizes = sizes
config = CustomBertConfig()


In [63]:
model = CustomBert(config)
# Optimizer
optimizer = AdamW(model.parameters(), lr=7e-4, weight_decay=0.01)

# Define the dataloader
trainloader = DataLoader(trainset, batch_size=200, shuffle=True)
valloader = DataLoader(valset, batch_size = 200, shuffle = True)
# Learning rate scheduler with warmup
total_steps = len(trainloader) * EPOCHS * 4
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0.1 * total_steps,
                                            num_training_steps=total_steps)
loss_function = nn.CrossEntropyLoss()

# Print the total number of model's parameters
num_params = sum(p.numel() for p in model.parameters())

print(f"Total number of parameters: {num_params}")

Total number of parameters: 20387099


In [ ]:
# Load the state dict from the file
state_dict = torch.load('BERT4RECmodel_weights.pth')

# Remove the unwanted key ('pos_enc.pe')
if "pos_enc.pe" in state_dict:
    del state_dict["pos_enc.pe"]

# Now load the modified state dict into the model
model.load_state_dict(state_dict)


<ipython-input-140-050f615bfb19>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('BERT4RECmodel_weights.pth')


<All keys matched successfully>

In [ ]:
EPOCHS = 16
# Initialize the GradScaler for gradient scaling
scaler = GradScaler()

for epoch in range(EPOCHS):
    # Initialize parameters for each epoch
    train_loss = 0
    train_batches = 0
    train_elems = 0
    train_correct = 0
    model.train() # Set model to training mode
    # Iterate over batches in the trainloader
    for batch_idx, batch in enumerate(trainloader):
        if batch_idx % 30 == 0:
            print(f"Training process: {batch_idx/len(trainloader)*100:.2f}%")

        mask_pred_output = []
        # Zero the gradients from the previous iteration
        optimizer.zero_grad()

        # Extract the required data
        mask_indices = batch[6]  # Indices to be masked
        train_target = batch[7]  # The target labels

        # Enable autocasting for mixed-precision
        with autocast():
            output = model(batch)  # Forward pass
            # Extract the representation of the masked indices
            #mask_pred_output = output[torch.arange(output.shape[0]), mask_indices, :]

            loss = loss_function(output, train_target).to(torch.float32)  # Compute loss

        # Backward pass with gradient scaling
        scaler.scale(loss).backward()

        # Update the model weights
        scaler.step(optimizer)
        scaler.update()

        # Accumulate metrics
        top_k = 5  # Top k predictions to consider
        _, topk_indices = torch.topk(output.data, top_k, dim=1, largest=True, sorted=True)

        # Check if the target is among the top k predictions for each example in the batch
        train_correct += (topk_indices == train_target.unsqueeze(1)).any(dim=1).sum().item()

        # Update the number of elements predicted
        train_elems += train_target.size(0)

        # Update the number of batches trained
        train_batches += 1

        # Accumulate loss for reporting later
        train_loss += loss.item()

        # Update scheduler for learning rate adjustment
        scheduler.step()

    # Compute average loss and accuracy for the epoch
    avg_train_loss = train_loss / train_batches
    train_acc = train_correct / train_elems

    # Print or log the results for this epoch
    print(f"Epoch {epoch+1}/{EPOCHS} - Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_acc:.4f}")

    # Evaluate the model on the test dataset.
    val_loss = 0.0
    val_correct = 0
    val_batches = 0
    val_elems = 0
    model.eval()  # Set model to eval mode for inference

    for batch_idx, batch in enumerate(valloader):
      if batch_idx %50 == 0:
        print(f" Validation Process: {batch_idx/len(valloader)*100:.4f}%")
      mask_indices = batch[6]
      val_target = batch[7]

      # Forward pass
      output = model(batch)

      #mask_pred_output = output[torch.arange(output.shape[0]), mask_indices, :]
      loss = loss_function(output, val_target)

      # Accumulate metrics
      top_k = 5
      _, topk_indices = torch.topk(output.data, top_k, dim=1, largest=True, sorted=True)
      val_correct += (topk_indices == val_target.unsqueeze(1)).any(dim=1).sum().item()
      val_elems += val_target.size(0)
      val_batches += 1
      val_loss += loss.item()

    avg_val_loss = val_loss / val_batches
    val_acc = val_correct / val_elems
    print(f"Epoch {epoch+1}/{EPOCHS} - Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")
    torch.save(model.state_dict(), 'BERT4RECmodel_weights.pth')


/var/folders/ry/1l9br0wd433fd3xsqyszbnlr0000gn/T/ipykernel_1557/1272403024.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/var/folders/ry/1l9br0wd433fd3xsqyszbnlr0000gn/T/ipykernel_1557/1272403024.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Training process: 0.00%
Training process: 12.24%
Training process: 24.49%
Training process: 36.73%
Training process: 48.98%
Training process: 61.22%
Training process: 73.47%
Training process: 85.71%
Training process: 97.96%
Epoch 1/16 - Training Loss: 9.9319, Training Accuracy: 0.0062
 Validation Process: 0.0000%
Epoch 1/16 - Validation Loss: 10.2585, Validation Accuracy: 0.0136
Training process: 0.00%
Training process: 12.24%
Training process: 24.49%
Training process: 36.73%
Training process: 48.98%
Training process: 61.22%
Training process: 73.47%
Training process: 85.71%
Training process: 97.96%
Epoch 2/16 - Training Loss: 9.8867, Training Accuracy: 0.0063
 Validation Process: 0.0000%
Epoch 2/16 - Validation Loss: 10.2909, Validation Accuracy: 0.0129
Training process: 0.00%
Training process: 12.24%
Training process: 24.49%
Training process: 36.73%
Training process: 48.98%
Training process: 61.22%
Training process: 73.47%
Training process: 85.71%
Training process: 97.96%
Epoch 3/16 -

I think a major problem is that the book_ids the model learns the embeddings for during training might not be asked during validation
In fact, I'm suspecting that the book_ids the mdoel knows about already are barely asked for during validation. therefore we actually need a lot more computational power to get through this. so how can we limit the number of the books? you are using a tokenizer that is adding sooooo many book ids. if we only use 10k users, we might actually be able to limit the number of unique book ids as well. ii

In [487]:
 # Evaluate the model on the test dataset.
    val_loss = 0.0
    val_correct = 0
    val_batches = 0
    val_elems = 0
    model.eval()  # Set model to eval mode for inference

    for batch_idx, batch in enumerate(valloader):
      if batch_idx %50 == 0:
        print(f" Validation Process: {batch_idx/len(valloader)*100:.4f}%")
      mask_indices = batch[6]
      val_target = batch[7]

      # Forward pass
      output = model(batch)

      #mask_pred_output = output[torch.arange(output.shape[0]), mask_indices, :]
      loss = loss_function(output, val_target)

      # Accumulate metrics
      top_k = 10
      _, topk_indices = torch.topk(output.data, top_k, dim=1, largest=True, sorted=True)
      val_correct += (topk_indices == val_target.unsqueeze(1)).any(dim=1).sum().item()
      val_elems += val_target.size(0)
      val_batches += 1
      val_loss += loss.item()

    avg_val_loss = val_loss / val_batches
    val_acc = val_correct / val_elems
    print(f"Epoch {epoch+1}/{EPOCHS} - Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")



IndentationError: unexpected indent (2600871397.py, line 2)

In [ ]:
torch.save(model.state_dict(), 'BERT4RECmodel_weights.pth')

In [ ]:
# if wanted to concatenate: # Repeat user embeddings across the sequence length
user_embeddings = user_embeddings.unsqueeze(1).repeat(1, book_embeddings.size(1), 1)  # (batch_size, len_seq, len_emb)

# Combine book embeddings with user embeddings (concatenate)
combined_embeddings = torch.cat([book_embeddings, user_embeddings], dim=-1)  # (batch_size, len_seq, len_emb*2)


In [ ]:
class CustomBert(nn.Module):
    def __init__(self, config):
        super(CustomBert, self).__init__()

        self.len_emb = config.hidden_size
        self.decay_rate = 0.1
        self.dropout = nn.Dropout(p=config.hidden_dropout_prob)  # Dropout layer
        self.norm1 = nn.LayerNorm(config.hidden_size)  # Layer Normalization
        self.norm2 = nn.LayerNorm(config.hidden_size)  # Layer Normalization
        self.norm3 = nn.LayerNorm(config.hidden_size)  # Layer Normalization
        self.norm4 = nn.LayerNorm(config.hidden_size)
        self.norm5 = nn.LayerNorm(config.hidden_size)
        self.norm6 = nn.LayerNorm(config.hidden_size)
        # Embedding layers for different features like ratings, train_input, and authors
        self.embedding_layers = nn.ModuleList([nn.Embedding(size, config.hidden_size) for size in config.sizes])

        # Linear layers for genre and description features
        self.linear_layers = nn.ModuleList([nn.Linear(384, config.hidden_size) for _ in range(2)])

        # Embedding layer for user_id
        self.embed_user = nn.Embedding(len(Tokenizer_user_id.word_index),config.hidden_size)

        # Positional encoding and dropout
        self.pos_enc = Pos_Enc()

        # Encoder to hold multiple layers
        self.encoder_layers = nn.ModuleList([EncoderLayer(config) for _ in range(config.num_enc_layers)])

        # Output layer
        self.linear = nn.Linear(config.hidden_size, config.item_size)

    def normalize_time(self, timestamps):
        """
        This function will normalize and create the recency parameter
        """
        # Ensure the timestamps are a tensor and properly handle gradient tracking
        timestamps = timestamps.clone().detach().float()
        # Calculate the max timestamp for each user's history (across each row)
        max_ts = timestamps.max(dim=1, keepdim=True)[0]  # max across each row (user's history)
        # Normalize the timestamps by dividing by the max timestamp for each row
        normalized_timestamps = timestamps / max_ts
        return normalized_timestamps

    def forward(self, input):
        """
        Input will be loaded by the dataloader and will be a tuple of values.
        """
        train_input = input[0]
        ratings = input[1]
        timestamps = input[2]
        authors = input[3]
        genre = input[4]
        description = input[5]
        user_ids = input[8]

        # Embedding layer for train_input, authors, and ratings
        combined_embed = self.norm4(sum([self.embedding_layers[i](x) for i, x in enumerate([train_input,authors, ratings])]))

        # Add the genre and description layers
        combined_embed += self.norm5(sum([self.linear_layers[i](x.float()) for i, x in enumerate([genre, description])]))

        # Map the user id to an embedding layer and add to the combined embedding
        user_embedded = self.norm6(self.embed_user(user_ids))
        user_embedded = user_embedded.unsqueeze(1).repeat(1, combined_embed.size(1), 1)  # (batch_size, len_seq, len_emb)
        combined_embed += user_embedded # combining user_id book_ids, and book features

        # Add positional encoding and normalize
        combined_embed = self.pos_enc(self.norm2(combined_embed)) * torch.sqrt(torch.tensor(self.len_emb))
        encoder_output = self.norm3(combined_embed)
        # Define attention mask (1 if not zero, 0 if zero)
        attention_mask = (train_input != 0).unsqueeze(1).unsqueeze(2)  # size = (batch_size, 1, 1, len_seq)
        print(f"dim of the mask is: {attention_mask.shape}")
        # Normalize the timestamps
        timestamp_norm = self.normalize_time(timestamps)

        # Pass through the encoder layers (6 times as defined)
        for encoder_layer in self.encoder_layers:
          # Pass through the encoder layer
          encoder_output = encoder_layer(encoder_output, timestamp_norm, attention_mask)

          # Apply dropout after each encoder layer's output
          encoder_output = self.dropout(encoder_output)

          # Apply layer normalization
          encoder_output = self.norm1(encoder_output)

        # Final output layer
        final_output = self.linear(encoder_output)

        return final_output
